# Unlimited Token Stable Diffusion XL with noobai-XL-1.1

**Author:** Vladus-CPU  
**Last Updated:** 2025-01-01 22:03:57 UTC

This notebook provides implementation of Stable Diffusion XL with unlimited token length for prompts using the noobai-XL-1.1 model.

In [ ]:
!pip install --upgrade diffusers transformers torch torchvision safetensors accelerate xformers matplotlib

In [ ]:
from diffusers import StableDiffusionXLPipeline, EulerAncestralDiscreteScheduler, DDIMScheduler
import torch
from PIL import Image
import matplotlib.pyplot as plt
import random
from datetime import datetime

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

In [ ]:
def load_pipeline_without_limits(scheduler_type="Euler"):
    """Load the noobai-XL-1.1 pipeline with unlimited token length."""
    model_id = "Laxhar/noobai-XL-1.1"
    
    pipe = StableDiffusionXLPipeline.from_pretrained(
        model_id,
        torch_dtype=torch.float16 if device.type == "cuda" else torch.float32,
        safety_checker=None,
        use_safetensors=True
    )
    
    # Remove token length limits
    if hasattr(pipe, 'tokenizer') and pipe.tokenizer is not None:
        pipe.tokenizer.model_max_length = int(1e9)
        if hasattr(pipe.tokenizer, 'max_length'):
            pipe.tokenizer.max_length = int(1e9)
            
    if hasattr(pipe, 'tokenizer_2') and pipe.tokenizer_2 is not None:
        pipe.tokenizer_2.model_max_length = int(1e9)
        if hasattr(pipe.tokenizer_2, 'max_length'):
            pipe.tokenizer_2.max_length = int(1e9)
    
    # Set scheduler
    if scheduler_type == "Euler":
        pipe.scheduler = EulerAncestralDiscreteScheduler.from_config(pipe.scheduler.config)
    elif scheduler_type == "DDIM":
        pipe.scheduler = DDIMScheduler.from_config(pipe.scheduler.config)
    else:
        raise ValueError("Unknown scheduler type")
    
    # Optimizations
    if hasattr(pipe, "enable_xformers_memory_efficient_attention"):
        pipe.enable_xformers_memory_efficient_attention()
    
    if hasattr(pipe, "enable_vae_tiling"):
        pipe.enable_vae_tiling()
    
    # Device placement
    if device.type == "cuda" and torch.cuda.get_device_properties(0).total_memory < 12 * 1024 * 1024 * 1024:
        pipe.enable_model_cpu_offload()
    else:
        pipe = pipe.to(device)
    
    return pipe

In [ ]:
def generate_image(
    prompt,
    negative_prompt="",
    sampler="Euler",
    sampling_steps=30,
    seed=None,
    cfg_scale=7.0,
    image_size=(1024, 1024),
    hires_fix=False,
    hires_scale=1.5,
    hires_steps=20
):
    """Generate an image using noobai-XL-1.1 model with unlimited prompt length."""
    try:
        if seed is not None:
            torch.manual_seed(seed)
            random.seed(seed)
            generator = torch.Generator(device=device).manual_seed(seed)
        else:
            generator = torch.Generator(device=device).manual_seed(random.randint(0, 2**32 - 1))
        
        pipe = load_pipeline_without_limits(sampler)
        
        with torch.autocast(device.type):
            output = pipe(
                prompt=prompt,
                negative_prompt=negative_prompt,
                num_inference_steps=sampling_steps,
                guidance_scale=cfg_scale,
                height=image_size[1],
                width=image_size[0],
                generator=generator
            )
        
        image = output.images[0]
        
        if hires_fix:
            new_width = int(image_size[0] * hires_scale)
            new_height = int(image_size[1] * hires_scale)
            
            with torch.autocast(device.type):
                output = pipe(
                    prompt=prompt,
                    negative_prompt=negative_prompt,
                    num_inference_steps=hires_steps,
                    guidance_scale=cfg_scale,
                    height=new_height,
                    width=new_width,
                    generator=generator
                )
            
            image = output.images[0]
        
        if device.type == "cuda":
            torch.cuda.empty_cache()
        
        return image
    
    except Exception as e:
        print(f"Error generating image: {str(e)}")
        if device.type == "cuda":
            torch.cuda.empty_cache()
        raise

In [ ]:
# Test image generation with sample prompt
prompt = """
masterpiece, best quality, ultra detailed, A majestic dragon soaring through storm clouds,
lightning crackling around its wings, highly detailed scales showing intricate patterns and textures,
epic lighting with multiple lightning bolts, volumetric storm clouds
"""

negative_prompt = """
worst quality, low quality, normal quality, lowres, low details, oversaturated,
undersaturated, overexposed, underexposed, grainy, blur, blurry, text, watermark,
signature, error, drawing, painting, illustration, anime, cartoon, stylized
"""

params = {
    'negative_prompt': negative_prompt,
    'sampler': "Euler",
    'sampling_steps': 30,
    'seed': 42,
    'cfg_scale': 7.0,
    'image_size': (1024, 1024),
    'hires_fix': True,
    'hires_scale': 1.5,
    'hires_steps': 20
}

try:
    generated_image = generate_image(prompt, **params)
    
    plt.figure(figsize=(15, 15))
    plt.imshow(generated_image)
    plt.axis('off')
    plt.show()
    
    timestamp = datetime.utcnow().strftime("%Y%m%d_%H%M%S")
    output_path = f"generated_image_{timestamp}_seed{params['seed']}.png"
    generated_image.save(output_path)
    print(f"Image saved to {output_path}")
    print(f"Seed used: {params['seed']}")

except Exception as e:
    print(f"Failed to generate image: {str(e)}")